In [ ]:
#reload source files automatically
%load_ext autoreload
%autoreload 2

In [ ]:
import pandas as pd
import numpy as np
from tensorflow.keras.callbacks import EarlyStopping
from wtrec.loader import load_from_directory
from wtrec.transformer import transform_for_approach
from wtrec.splitter import split_fixed, split_random, series_to_sequence
from wtrec.classifier import create_nn_classifier
from wtrec.evaluator import evaluation_report

### Load raw data to dataframe

In [ ]:
raw = load_from_directory('./data/dataset_labelled/')

### Generate feature vectors for each sample in raw dataframe

In [ ]:
approach_df = transform_for_approach(raw, strategy='char_blocks', resize_mode='resize')

### Split dataset into test and train data according to previously computed indices

In [ ]:
train_X, train_Y, test_X, test_Y = split_fixed(approach_df, './data/split_idx.npz')

train_seq = series_to_sequence(train_X, train_Y, batch_size=64)
test_seq = series_to_sequence(test_X, test_Y, batch_size=64)

### Learn neural net model w/ epoch-wise evaluation

In [ ]:
earlyStopping = EarlyStopping(monitor='val_loss', patience=10, verbose=1, restore_best_weights=True)

clf = create_nn_classifier()
clf.fit_generator(train_seq,
                  epochs=40,
                  validation_data=test_seq,
                  callbacks=[earlyStopping],
                  validation_freq=1,
                  verbose=1)

In [ ]:
y_pred = np.eye(4, dtype=int)[np.argmax(clf.predict_generator(test_seq), axis=1)]
y_true = test_seq.y

evaluation_report(y_true, y_pred, classes=test_seq.classes)